In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

pd.set_option('display.float_format', lambda x: f'{x :.2f}')
from get_dataframes import GetDataframes
import hjson

import matplotlib.pyplot as plt
import explo_data_analysis.eda_movies as eda

with open("config.hjson") as fp:
    config = hjson.load(fp)

- Importation de la dataframe contenant seulement les acteurs.
- Cette dataframe a déjà été nettoyé comme indiqué précédemment.

In [2]:
datas = GetDataframes(config)
link = "actors_movies"
df = datas.get_dataframes(
    link,
    True
)

2023-11-07 16:35:11 INFO     TSV files already exist.
2023-11-07 16:35:11 INFO     Parquet loaded ! Importing actors_movies...
2023-11-07 16:35:12 INFO     Updating...
2023-11-07 16:35:12 INFO     Parquet loaded ! Importing actors...
2023-11-07 16:35:36 INFO     Parquet loaded ! Importing persons...
2023-11-07 16:35:56 INFO     Parquet loaded ! Importing movies_cleaned...
2023-11-07 16:35:56 INFO     Values modified, creating new cleaned movies...
2023-11-07 16:35:56 INFO     Parquet loaded ! Importing movies...
2023-11-07 16:35:56 INFO     Dataframe movies_cleaned ready to use!
2023-11-07 16:35:56 INFO     Parquet loaded ! Importing movies_cleaned...
2023-11-07 16:36:55 INFO     Replace tt by movies titles...
2023-11-07 16:36:55 INFO     Writing actors_movies dataframe...
2023-11-07 16:36:56 INFO     Dataframe actors_movies ready to use!


- On veut afficher quels acteurs ont joué dans le plus de films pour chaque décénnie.

In [3]:
grouped_df = df.groupby(
    ['cuts', 'person_name'],
    observed=True
).size().reset_index(name='total_film_acteurs').sort_values(by="total_film_acteurs")

top_acteurs_decennie = grouped_df.groupby(
    'cuts',
    observed=True
).apply(lambda x: x.nlargest(1, 'total_film_acteurs')).reset_index(drop=True)

decennies = top_acteurs_decennie['cuts']
noms_acteurs = top_acteurs_decennie['person_name']
nombre_films = top_acteurs_decennie['total_film_acteurs']

# plt.figure(figsize=(16, 8))
# bars = plt.barh(
#     decennies,
#     nombre_films,
#     color='#66cdaa',
#     edgecolor="black"
# )
# for i, (count, name) in enumerate(zip(
#     top_acteurs_decennie['total_film_acteurs'],
#     top_acteurs_decennie['person_name'])):
#     plt.text(0.5, i, name, color='black', va='center', ha='left', fontweight='bold')

# plt.title('Acteur N°1 par Décennie et Nombre de Films Joués')
# plt.xlabel('Nombre de Films Joués')
# plt.ylabel('Décennie')
# plt.show()

fig = go.Figure(data=[
    go.Bar(
        x=nombre_films,
        y=decennies,
        orientation='h',
        marker=dict(
            color='#66cdaa',
            line=dict(
                color='black',
                width=1
            )
        ),
        text=noms_acteurs,
        textposition='auto',
        width=1,
        textfont=dict(
            size=14,
            color='black'
        )
    )
])

fig.update_layout(
    title='Acteur N°1 par Décennie et Nombre de Films Joués',
    xaxis_title='Nombre de Films Joués',
    yaxis_title='Décennie'
)

fig.show()


- On veut afficher quels acteurs ont joué dans le plus de films pour chaque genre.

In [4]:
actors_by_genre = df.explode('titre_genres').groupby(['person_name', 'titre_genres']).size().reset_index(name='count')
top_actors_by_genre = actors_by_genre.sort_values('count', ascending=False).drop_duplicates('titre_genres')[:10][::-1]

# plt.figure(figsize=(20, 10))
# plt.barh(
#     top_actors_by_genre['titre_genres'],
#     top_actors_by_genre['count'],
#     color='#006b3c',
#     edgecolor = "black",
# )
# plt.xlabel('Nombre de Films')
# plt.ylabel('Genres')
# plt.title('Acteurs les plus fréquemment associés aux top 10 des genres')

# for i, (count, name) in enumerate(zip(
#     top_actors_by_genre['count'],
#     top_actors_by_genre['person_name'])):
#     plt.text(1, i, name, color='white', va='center', ha='left', fontweight='bold')


# plt.show()

fig = go.Figure(data=[
    go.Bar(
        x=top_actors_by_genre['count'],
        y=top_actors_by_genre['titre_genres'],
        orientation='h',
        marker=dict(
            color='#006b3c',
            line=dict(
                color='black',
                width=1
            )
        ),
        text=top_actors_by_genre['person_name'],
        textposition='auto',
        width=1,
        textfont=dict(
            size=14,
            color='white'
        )
    )
])

fig.update_layout(
    title='Acteurs les plus fréquemment associés aux top 10 des genres',
    xaxis_title='Nombre de Films',
    yaxis_title='Genres'
)

fig.show()



- On veut afficher quels acteurs ont joué dans le plus de films au total.

In [5]:
actors_film_count = df.groupby('person_name').size().reset_index(name='film_count')

top_actors_film_count = actors_film_count.sort_values('film_count', ascending=False).head(10)[::-1]

# plt.figure(figsize=(20, 10))
# plt.barh(
#     top_actors_film_count['person_name'],
#     top_actors_film_count['film_count'],
#     color='#daa520',
#     edgecolor="black",
# )
# for i, (count, name) in enumerate(zip(
#     top_actors_film_count['person_name'],
#     top_actors_film_count['film_count'])):
#     plt.text(1, i, name, color='black', va='center', ha='left', fontweight='bold')

# plt.xlabel('Nombre de Films')
# plt.ylabel('Acteurs')
# plt.title('Acteurs ayant joué dans le plus grand nombre de films')
# plt.show()

fig = go.Figure(data=[
    go.Bar(
        x=top_actors_film_count['film_count'],
        y=top_actors_film_count['person_name'],
        orientation='h',
        marker=dict(
            color='#daa520',
            line=dict(
                color='black',
                width=1
            )
        ),
        text=top_actors_film_count['film_count'],
        textposition='inside',
        width=1,
        textfont=dict(
            size=14,
            color='black'
        )
    )
])

fig.update_layout(
    title='Acteurs les plus fréquemment associés aux top 10 des genres',
    xaxis_title='Nombre de Films',
    yaxis_title='Genres'
)

fig.show()

- On veut afficher les acteurs ainsi que le nombre de vote total qu'on reçu les films dans lesquels ils ont joué.

In [6]:
actors_by_votes = df.groupby('person_name')['rating_votes'].sum().reset_index()

top_actors_by_votes = actors_by_votes.sort_values('rating_votes', ascending=False).head(10)[::-1]
# plt.figure(figsize=(20, 10))
# plt.barh(
#     top_actors_by_votes['person_name'],
#     top_actors_by_votes['rating_votes'],
#     color='#008080',
#     edgecolor = "black"
# )

# plt.xlabel('Total des votes')
# plt.ylabel('Acteurs')
# plt.title('Top 10 des acteurs dans des films ayant eu le plus de votes')
# plt.ticklabel_format(style='plain', axis='x')
# plt.show()


fig = go.Figure(data=[
    go.Bar(
        x=top_actors_by_votes['rating_votes'],
        y=top_actors_by_votes['person_name'],
        orientation='h',
        marker=dict(
            color='#daa520',
            line=dict(
                color='black',
                width=1
            )
        ),
        text=top_actors_by_votes['rating_votes'],
        textposition='auto',
        width=1,
        textfont=dict(
            size=14,
            color='black'
        )
    )
])

fig.update_layout(
    title='Top 10 des acteurs dans des films ayant eu le plus de votes',
    xaxis_title='Acteurs',
    yaxis_title='Total des votes'
)

fig.show()
